# WSI exploratory check

Load a whole slide image from `/home/khdp-user/workspace/dataset/Slide`, report magnification metadata, and validate the MONAI WSI preprocessing pipeline used by this repo (LoadROId + tile generation).

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import monai
import numpy as np
from monai.data.wsi_reader import WSIReader
from openslide import OpenSlide

from kidpro.preprocessing.data.create_tiles_dataset import generate_tiles
from kidpro.preprocessing.data.foreground_segmentation import LoadROId

print("monai version:", monai.__version__)

In [ ]:
slide_dir = Path("/home/khdp-user/workspace/dataset/Slide")
if not slide_dir.exists():
    raise FileNotFoundError(f"Slide directory does not exist: {slide_dir}")

wsi_exts = {".svs", ".tif", ".tiff", ".ndpi", ".mrxs", ".scn", ".vms", ".vmu"}
slide_paths = sorted([p for p in slide_dir.iterdir() if p.is_file() and p.suffix.lower() in wsi_exts])
if not slide_paths:
    slide_paths = sorted([p for p in slide_dir.iterdir() if p.is_file()])

if not slide_paths:
    raise FileNotFoundError(f"No slide files found under {slide_dir}")

slide_path = slide_paths[0]
print("Selected slide:", slide_path)
slide_path

In [ ]:
wsi = OpenSlide(str(slide_path))
props = dict(wsi.properties)

objective_power = props.get("openslide.objective-power") or props.get("aperio.AppMag")
mpp_x = props.get("openslide.mpp-x")
mpp_y = props.get("openslide.mpp-y")

print("Level count:", wsi.level_count)
print("Level dimensions:", wsi.level_dimensions)
print("Level downsamples:", wsi.level_downsamples)
print("Objective power:", objective_power)
print("MPP (x, y):", mpp_x, mpp_y)

if objective_power is not None:
    try:
        base_mag = float(objective_power)
        level_mags = [base_mag / ds for ds in wsi.level_downsamples]
        print("Estimated magnification per level:", level_mags)
    except ValueError:
        print("Objective power is not numeric; raw value:", objective_power)

wsi.close()

In [ ]:
reader = WSIReader(backend="OpenSlide")
slide_obj = reader.read(str(slide_path))
level = 1 if slide_obj.level_count > 1 else 0
level_dim = slide_obj.level_dimensions[level]

preview_size = (min(512, level_dim[1]), min(512, level_dim[0]))  # (H, W)
preview, preview_meta = reader.get_data(slide_obj, level=level, size=preview_size)
preview_arr = np.asarray(preview)

print("Preview shape (C, H, W):", preview_arr.shape)
print("Preview meta keys:", list(preview_meta.keys())[:10])

slide_obj.close()

In [ ]:
margin = 32
tile_size = 256
occupancy_threshold = 0.6
hsv_s_threshold = 0.05

sample = {"slide_id": slide_path.stem, "image": str(slide_path)}
loader = LoadROId(
    WSIReader(backend="OpenSlide"),
    level=level,
    margin=margin,
    foreground_threshold=None,
    hsv_s_threshold=hsv_s_threshold,
)

sample = loader(sample)
img = np.asarray(sample["image"])
print("Loaded ROI image shape (C, H, W):", img.shape, "dtype:", img.dtype)
print("Foreground threshold:", sample["foreground_threshold"])
print("Origin (level-0):", sample["origin"], "Scale:", sample["scale"])

tiles, coords = generate_tiles(
    img,
    tile_size=tile_size,
    foreground_threshold=sample["foreground_threshold"],
    occupancy_threshold=occupancy_threshold,
    hsv_s_threshold=hsv_s_threshold,
)
print("Tiles shape:", tiles.shape)
print("Tile coords shape:", coords.shape)

In [ ]:
import matplotlib.patches as patches

if coords.size == 0:
    print("No tiles to overlay (coords is empty).")
else:
    img_hwc = np.moveaxis(img, 0, -1)

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img_hwc)
    for x, y in coords:
        rect = patches.Rectangle(
            (x, y), tile_size, tile_size,
            linewidth=1, edgecolor="yellow", facecolor="none", alpha=0.4
        )
        ax.add_patch(rect)
    ax.set_title(f"Overlayed {coords.shape[0]} tiles on ROI")
    ax.axis("off")

In [ ]:
import inspect

try:
    import lazyslide as zs
    import wsidata
    from wsidata import open_wsi
except ImportError as exc:
    raise ImportError(
        "lazyslide/wsidata not available; install to run lazyslide checks"
    ) from exc

print("lazyslide version:", getattr(zs, "__version__", "unknown"))
print("wsidata version:", getattr(wsidata, "__version__", "unknown"))
print("tile_tissues signature:", inspect.signature(zs.pp.tile_tissues))
print("tile_tissues doc (head):")
print("\n".join((zs.pp.tile_tissues.__doc__ or "").splitlines()[:15]))

In [ ]:
wsi_zs = open_wsi(str(slide_path))

# Optional: find tissues before tiling (required for tissue-based tiling)
zs.pp.find_tissues(wsi_zs)

tiles_512 = zs.pp.tile_tissues(
    wsi_zs,
    tile_px=512,
    mpp=0.25,
    key_added="tiles_512",
    return_tiles=True,
)

tiles_256 = zs.pp.tile_tissues(
    wsi_zs,
    tile_px=256,
    mpp=0.5,
    key_added="tiles_256",
    return_tiles=True,
)

# Return structure can vary by version; try to count tiles safely

def _tiles_count(tiles_ret):
    if tiles_ret is None:
        return None
    if isinstance(tiles_ret, tuple) and len(tiles_ret) > 0:
        try:
            return len(tiles_ret[0])
        except TypeError:
            return None
    try:
        return len(tiles_ret)
    except TypeError:
        return None

print("tiles_512 count:", _tiles_count(tiles_512))
print("tiles_256 count:", _tiles_count(tiles_256))
print("tiles keys:", list(wsi_zs.shapes.keys()))

In [ ]:
def _read_tile_any(wsi, tile_key, tile_id):
    tile_spec = wsi.tile_spec(tile_key) if hasattr(wsi, "tile_spec") else None
    if tile_spec is not None:
        for method in ("read_tile", "get_tile", "read_tiles"):
            if hasattr(tile_spec, method):
                fn = getattr(tile_spec, method)
                try:
                    return fn(wsi, tile_id)
                except TypeError:
                    try:
                        return fn(tile_id)
                    except Exception:
                        pass
    for method in ("read_tile", "get_tile"):
        if hasattr(wsi, method):
            fn = getattr(wsi, method)
            try:
                return fn(tile_id, key=tile_key)
            except TypeError:
                try:
                    return fn(tile_id)
                except Exception:
                    pass
    raise RuntimeError("No tile read method found; inspect tile_spec/wsi methods")

print("tile_spec tiles_512:", wsi_zs.tile_spec("tiles_512"))
print("tile_spec tiles_256:", wsi_zs.tile_spec("tiles_256"))
print("tile_spec methods:", [m for m in dir(wsi_zs.tile_spec("tiles_512")) if "tile" in m])

In [ ]:
from PIL import Image

out_dir = Path("saved_tiles_512")
out_dir.mkdir(parents=True, exist_ok=True)

tiles_gdf = wsi_zs.shapes["tiles_512"].copy()
if "tile_id" in tiles_gdf.columns:
    tile_ids = tiles_gdf["tile_id"].tolist()
else:
    tile_ids = tiles_gdf.index.tolist()

saved = 0
for tile_id in tile_ids[:5]:
    tile = _read_tile_any(wsi_zs, "tiles_512", tile_id)
    tile_arr = np.asarray(tile)
    if tile_arr.ndim == 3 and tile_arr.shape[0] in (1, 3, 4):
        tile_arr = np.moveaxis(tile_arr, 0, -1)
    tile_img = Image.fromarray(tile_arr.astype(np.uint8))
    tile_img.save(out_dir / f"tile_{tile_id}.png")
    saved += 1

print(f"Saved {saved} tiles to {out_dir}")